In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from IPython.html.widgets import *
from IPython.html.widgets import interact, interactive, fixed
from IPython.html import widgets
from IPython.display import display

//anaconda3/lib/python3.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
standardizer = lambda x: (x-np.min(x))/(np.max(x)-np.min(x))
seis_vlm = np.load('./F3_small_seis.npy')
seis_vlm = standardizer(seis_vlm)
fltprob_vlm = np.load('./F3_small_pred.npy')

In [3]:
def CreateImgAlpha(img_input):
    img_alpha = np.zeros([np.shape(img_input)[0], np.shape(img_input)[1], 4])
    threshold = 0.9
    img_input[img_input < threshold] = 0
    # Yellow: (1,1,0), Red: (1,0,0)
    img_alpha[:,:,0] = 1
    img_alpha[:,:,1] = 0
    img_alpha[:,:,2] = 0
    img_alpha[...,-1] = img_input
    return img_alpha

def slice_to_show(idx, seis_vlm, fltprob_vlm, slice_select):
    if   slice_select == 'x-slice':
        r_slice = seis_vlm[idx,:,:]
        l_slice = fltprob_vlm[idx,:,:]        
    elif slice_select == 'y-slice':
        r_slice = seis_vlm[:,idx,:]
        l_slice = fltprob_vlm[:,idx,:]
    else:
        r_slice = seis_vlm[:,:,idx].T
        l_slice = fltprob_vlm[:,:,idx].T
    return r_slice, l_slice

def plt_slice(val, slice_select, xlim, ylim):
    r_slice, l_slice = slice_to_show(val[0], seis_vlm, fltprob_vlm, slice_select)
    fig, ax = plt.subplots()
    plt.imshow(r_slice.T, cmap = plt.cm.gray_r, vmin=0.1, vmax=0.9, aspect = 'equal')
    plt.imshow(CreateImgAlpha(l_slice.T), alpha=0.5, aspect = 'equal')
    plt.plot([val[1], val[1]],[0, ylim[1]], lw = 1, color = 'magenta')
    plt.plot([0, xlim[1]],[val[2], val[2]], lw = 1, color = 'magenta')
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.invert_yaxis()
    if   slice_select == 'x-slice':
        ax.set_xlabel('y')
        ax.set_ylabel('z')
    elif slice_select == 'y-slice':
        ax.set_xlabel('x')
        ax.set_ylabel('z')
    else:
        ax.set_xlabel('y')
        ax.set_ylabel('x')
    plt.title(slice_select)
    scaler = 40
    ax.figure.set_size_inches(xlim[1]/scaler, ylim[1]/scaler)
    plt.show()
    return
    
def wrapper_slice(x, y, z, slice_select):
    if slice_select == 'x-slice':
        val = (x, y, z)
        xlim = [0, ny]
        ylim = [0, nz]        
    elif slice_select == 'y-slice':
        val = (y, x, z)
        xlim = [0, nx]
        ylim = [0, nz]
    else:
        val = (z, y, x)
        xlim = [0, ny]
        ylim = [0, nx]
    display(plt_slice(val, slice_select, xlim, ylim))
        
nx = seis_vlm.shape[0]
ny = seis_vlm.shape[1]
nz = seis_vlm.shape[2]
interact(wrapper_slice, x = IntSlider(min=0, max=nx-1,step=1,value=int(nx/2)),
                        y = IntSlider(min=0, max=ny-1,step=1,value=int(ny/2)),
                        z = IntSlider(min=0, max=nz-1,step=1,value=int(nz/2)),
                        slice_select = ['x-slice','y-slice','z-slice'])

interactive(children=(IntSlider(value=64, description='x', max=127), IntSlider(value=64, description='y', max=…

<function __main__.wrapper_slice(x, y, z, slice_select)>